<a href="https://colab.research.google.com/github/Xw-Jia/2019_SaveYourself_GetOffer/blob/master/%E4%B9%A6%E7%9B%AE%E6%95%B4%E7%90%86/Python%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0/Part2%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E5%AE%9E%E8%B7%B5/%E4%BB%A3%E7%A0%81/6_1_using_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install numpy==1.16.2

+ one-hot：稀疏/高维/二进制向量
+ 词嵌入：密集/低维/浮点数向量/通过学习得到
+ 获得词嵌入的两种方法：
    - 通过完成主任务的同时学习词嵌入，开始时时随机的词向量，然后对这些词向量进行学习
    - 在不同的待解决的问题上预训练词嵌入，然后加载到模型中

In [8]:
# 不同的语种，不同的领域，关注的词向量的重点方面都不同，所以，最好，为特定任务训练词向量
import keras
keras.__version__

'2.2.4'

In [9]:
# 将一个Embedding层实例化
from keras.layers import Embedding
embedding_layer = Embedding(1000, 64)
# Embedding的两个参数：1000:最大单词索引+1； 64：维度
# Embedding实际是一个字典查找，单词索引-->Embedding-->对应的词向量
'''
Embedding层可以处理长度可变的序列，但是，在一个batch中，所有samples的长度必须相同
'''

'\nEmbedding层可以处理长度可变的序列，但是，在一个batch中，所有samples的长度必须相同\n'

应用到IMDB二分类：
1. 限制为前10000个最常见的word
2. sequence的长度限制为20
3. 每个word学习一个8维词嵌入
4. 词嵌入学习完毕是3维tensor，展成二维？？
5. 训练一个Dense层用于分类

In [0]:
from keras.datasets import imdb
from keras import preprocessing

# 作为特征的单词个数
max_features = 10000
# 截断补齐
maxlen = 20

# 数据加载为整数列表
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 整数list转成 `(samples, maxlen)`形式的整数tensor
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [11]:
'''
使用Embedding层和分类器
'''
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
# 指定Embedding的最大输入长度，方便Flatten
model.add(Embedding(10000, 8, input_length=maxlen))
# Embedding的输出形状 `(samples, maxlen, 8)`.

# 3D展平为：2D tensor`(samples, maxlen * 8)`
model.add(Flatten())

# 添加分类器
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 93us/step - loss: 0.6759 - acc: 0.6050 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 2s 85us/step - loss: 0.5657 - acc: 0.7427 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 2s 84us/step - loss: 

- 以上只有一个Dense层，没有作为序列来进行训练，所以会导致没办法考虑单词之间的关系和句子结构
- 可以使用CNN或RNN学到更多语序结构，序列整体的特征

In [12]:
'''
使用预训练的词嵌入
当可用数据很少，可以从语序连的嵌入空间中加载向量，预训练得到的是一般特征
GloVe：基于词共现矩阵进行因式分解
W2V：无监督计算低维密集向量
'''
#下载原始IMDB数据集，而不是使用Keras中预先分词的IMDB
from google.colab import files
uploaded = files.upload()

Saving imdb.csv to imdb.csv


In [15]:
!bash whereis imdb.csv


/usr/bin/whereis: /usr/bin/whereis: cannot execute binary file


In [20]:
# 需要处理标签，没搞成
import os

imdb_dir = 'content/imdb.csv'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

FileNotFoundError: ignored

In [21]:
'''
对数据分词，划分训练集和验证集
可以将数据限定为200个，使用预训练的词向量
'''

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  #序列长度，100
training_samples = 200  # 200训练
validation_samples = 10000  # 10000验证
max_words = 10000  # 前10000个最常见的词

'''
创建分词器，构建单词索引
将字符串转成整数index组成的list
'''
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# 划分之前，先要打乱数据，因为原来有序：(all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 0 unique tokens.
Shape of data tensor: (0, 100)
Shape of label tensor: (0,)


In [0]:
'''
下载GloVe词嵌入文件，（822MB-100维），解压
解析解压后的文件，构建将word映射为vector的索引
'''
#解析GloVe文件
glove_dir = '/home/ubuntu/data/'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [0]:
'''
构建一个加载到embedding层的嵌入矩阵
形状(max_words, embedding_dim)
其中，索引0不代表任何元素，只是占位符
'''
#准备GloVe词嵌入矩阵

embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
            
#定义模型
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

#加载GloVe嵌入
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False #冻结embedding层

#编译并训练模型
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

In [0]:
#画图
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
'''
对测试集数据进行分词
'''
test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

'''
在测试集上evaluate
'''
model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)